db = mongoConnection['TestLoop_RawData_MP']
print(db.DATA_To_MPHERE.count())
print(db['Time_Last_Extracted'].count())

#db['Time_Last_Extracted'].drop()
#db['DATA_To_MPHERE'].drop()

In [ ]:
# import data till

End_timestamp = dt.strptime("2020-07-12", '%Y-%m-%d')

In [2]:
import logging
from urllib.parse import urlparse
import time
import matplotlib.pyplot as plt
from gql import Client, gql
from gql.transport.requests import RequestsHTTPTransport
import requests
import pandas as pd
from datetime import datetime as dt, timedelta as td
from pymongo import MongoClient
from GetMacIDList import IQNet_MacId_list # will get us the list of macids connected to iqunet server
#from GraphQL_Client import GraphQLClient
logging.basicConfig(filename='Raw_to_Mongodb.log',level=logging.DEBUG)
logging.info('Importing raw data from Iqnet to MongoDB {}'.format(dt.now()))

In [2]:
class GraphQLClient(object):
    CONNECT_TIMEOUT = 15  # [sec]
    RETRY_DELAY = 10  # [sec]
    MAX_RETRIES = 3  # [-]

    class Decorators(object):
        @staticmethod
        def autoConnectingClient(wrappedMethod):
            def wrapper(obj, *args, **kwargs):
                for retry in range(GraphQLClient.MAX_RETRIES):
                    try:
                        return wrappedMethod(obj, *args, **kwargs)
                    except Exception:
                        pass
                    try:
                        obj._logger.warning(
                                '(Re)connecting to GraphQL service.'
                        )
                        obj.reconnect()
                    except ConnectionRefusedError:
                        obj._logger.warn(
                            'Connection refused. Retry in 10s.'.format(
                                GraphQLClient.RETRY_DELAY
                            )
                        )
                        time.sleep(GraphQLClient.RETRY_DELAY)
                else:  # So the exception is exposed.
                    obj.reconnect()
                    return wrappedMethod(obj, *args, **kwargs)
            return wrapper

    def __init__(self, serverUrl):
        self._logger = logging.getLogger(self.__class__.__name__)
        self.connect(
            serverUrl.geturl()
        )

    def __enter__(self):
        self.connect(
            serverUrl.geturl()
        )
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self._client = None

    def connect(self, url):
        host = url.split('//')[1].split('/')[0]
        request = requests.get(url,
                               headers={
                                       'Host': str(host),
                                       'Accept': 'text/html',
                                       }
                               )
        request.raise_for_status()
        csrf = request.cookies['csrftoken']
        self._client = Client(
                transport=RequestsHTTPTransport(url=url,
                                                cookies={"csrftoken": csrf},
                                                headers={'x-csrftoken':  csrf}
                                                ),
                fetch_schema_from_transport=True
                )

    def disconnect(self):
        self._client = None

    def reconnect(self):
        self.disconnect()
        self.connect(
            serverUrl.geturl()
        )

    @Decorators.autoConnectingClient
    def execute_query(self, querytext):
        query = gql(querytext)
        return self._client.execute(query)

In [41]:
class DataAcquisition(object):
    LOGGER = logging.getLogger('DataAcquisition')

    @staticmethod
    def get_sensor_data(serverUrl, macId, starttime, endtime, limit, axis,last_extract):
        with GraphQLClient(serverUrl) as client:
            querytext = '''
            { deviceManager { device(macId:"''' + macId + '''") {
                __typename
                ... on GrapheneVibrationCombo {vibrationTimestampHistory(start:"''' + str(starttime) + '''", end:"''' + str(endtime) + '''", limit:''' + str(limit) + ''', axis:"''' + axis + '''")}
            }}}
            '''
            #print('1st one : ' + querytext)
            result = client.execute_query(querytext)
            #print(result.keys())
            times = \
                result['deviceManager']['device']['vibrationTimestampHistory']
            dates, values, franges = ([], [], [])
            for t in times:
                #print(t)
                if len(t) == 32:
                    Iqnet_time = dt.strptime(t[:-13],'%Y-%m-%dT%H:%M:%S') + td(hours=4) # adding 4 hours to iqnet time coz of timezone
                elif len(t) == 25:
                    Iqnet_time = dt.strptime(t[:-6],'%Y-%m-%dT%H:%M:%S') + td(hours=4)
                else:
                    continue
                if (Iqnet_time >= last_extract):
                    #print(Iqnet_time)
                    result = DataAcquisition.get_sensor_measurement(
                        client,
                        macId,
                        t
                    )
                    dates.append(t)
                    deviceData = result['deviceManager']['device']
                    values.append(
                            deviceData['vibrationArray']['rawSamples']
                    )
                    franges.append(
                            deviceData['vibrationArray']['formatRange']
                    )
            return (values, dates, franges)
    
    @staticmethod
    def get_sensor_measurement(client, macId, isoDate):
        querytext = '''
        { deviceManager { device(macId:"''' + macId + '''") {
        __typename
        ... on GrapheneVibrationCombo { vibrationArray(isoDate: "''' + isoDate + '''") {
        numSamples rawSamples sampleRate formatRange axis }}
        }}}
        '''
        #print('2nd one : '+querytext)
        return client.execute_query(querytext)

In [4]:
try:
    mongoConnection = MongoClient('mongodb://localhost:27017/')
    logging.info('Successfully connected to MongoDB')
except:
    logging.warning('Could not connect to MongoDB Database!!!')
# Switch to Existing Database named siemensDubai_iQunet_Processed_Database

db = mongoConnection['TestLoop_RawData_MP']
Datacollection = db['DATA_To_MPHERE']
DF_Sensor_Loc = db['Time_Last_Extracted']

In [44]:
# save timestamp of the last value extracted from iqnet for a specific sensor
def getDateTimeforMacId(macId):
    global localDatabaseConnection,installationDate
    # Creat Collection, its similar to tables in RDBMS
    db = mongoConnection['TestLoop_RawData_MP']
    DF_Sensor_Loc = db['Time_Last_Extracted']
    if DF_Sensor_Loc.count_documents({'macId':macId})>0:    
        myPrecious = DF_Sensor_Loc.find_one({'macId':macId})
    else:
        myPrecious={
                "macId": macId,
                "MStartDate": dt.strptime("2020-07-10 00:00:00", '%Y-%m-%d %H:%M:%S')
                }
        DF_Sensor_Loc.insert_one(myPrecious)
#    print('\n',dataType,myPrecious.get('MStartDate'))
    return myPrecious.get('MStartDate')

In [10]:
def updateDateTimeforMacId(macId,Next_startDate):
    global localDatabaseConnection
    #localCollection = localDatabaseConnection.iQunet_Macid_DatesToStart
    db = mongoConnection['TestLoop_RawData_MP']
    DF_Sensor_Loc = db['Time_Last_Extracted']
    dateToUpdate=Next_startDate+td(seconds=1)
    myquery = { "macId": macId }
    newvalues = { "$set": { "MStartDate": dateToUpdate } }
    DF_Sensor_Loc.update_one(myquery, newvalues)  

db['New_raw_data'].remove({"Sensor_Loc":"MER-ED3"})

testiing

def check_entry_exits(row):
    DF_Sensor_Loc = db['Mac_Fault_list']
    try:
        if(DF_Sensor_Loc.find({ "Entry Date":     row["Entry Date"],                         \
                           "Asset-Id":row["Asset-Id"], \
                           "Sensor Location":row["Sensor Location"],\
                           "Macid":row["Macid"],
                         "Start Time":  row["Start Time"], \
                         "End Time": row["End Time"]}).count() == 0):
            DF_Sensor_Loc.insert_one(row.to_dict())
            return True # data does not exits hence insert and data will be loaded with no duplicates
        else:
            #print("Found entry")
            return False # As the data already exits raw data will not be loaded
    except ValueError:
            return False # skip entry when there is any data missing

In [14]:
# Ip address of the server
serverIP = "192.168.x.x"
serverUrl = urlparse('http://{:s}:8000/graphql'.format(serverIP))

#macIds = ['4c:92:10:8c']
macIds = IQNet_MacId_list()  # function to get the list of macids configured in iqnet server

limit = 200  # limit limits the number of returned measurements.  sample time per hour into hours 6 * 24
axis_list = ['X','Y','Z'] # axis allows to select data from only 1 or multiple axes

In [15]:
col_list2 = ['Vsample' + str(x) for x in range(1,4097)]
col_list1 =['Data_pull_time','Mac_Id','Axis']
columns = col_list1+ col_list2

In [46]:
# new code to extract raw data and save the time of last value extracted and to start from there next time
for Id in list(macIds):
    data = []
    for axis in axis_list:
        last_extract_date = getDateTimeforMacId(Id)
        Start_date = last_extract_date.date()
        End_Date = End_timestamp.date() # testing end date
        logging.info('{} {} : Data extraction between {} and {}'.format(Id,axis,last_extract_date,End_Date))
        try:
            (values, dates, franges) = DataAcquisition.get_sensor_data(serverUrl=serverUrl,macId=Id,starttime=Start_date,endtime=End_Date,limit=limit,axis=axis,last_extract=last_extract_date)
        except KeyError:
            logging.warning('No data available for Macid : {} axis : {}'.format(Id,axis))
            continue
        line = []
        for i in range(len(franges)):
            values[i] = [d/512.0*franges[i] for d in values[i]]
            if len(dates[i]) == 32:
                Iq_time = dt.strptime(dates[i][:-13],'%Y-%m-%dT%H:%M:%S') + td(hours=4) # adding 4 hours to iqnet time coz of timezone
            elif len(dates[i]) == 25:
                Iq_time = dt.strptime(dates[i][:-6],'%Y-%m-%dT%H:%M:%S') + td(hours=4)
            else:
                continue
            line.append(Iq_time)
            line.append(Id)
            line.append(axis)
            line.extend(values[i])
            data.append(line)  
            line=[]     
    if len(data) > 0:
        if len(data[0]) != 4099:
            continue
    df = pd.DataFrame(data,columns=columns)
    df.reset_index(inplace=True,drop=True)
    max_date = df.Data_pull_time.max()
    data_dict = df.to_dict("records")
    if len(data_dict) > 0 and df.shape[1] == 4099:
        db['DATA_To_MPHERE'].insert_many(data_dict)
        updateDateTimeforMacId(Id,max_date)
        logging.info('For Mac Id {} {} rows imported, imported till {}'.format(Id,df.shape[0],max_date))
    else:
        updateDateTimeforMacId(Id,End_timestamp)
        logging.info('No Data Available for Mac Id {}'.format(Id))
    #df.to_excel(Id.replace(":","")+"_TEST_LOOP_RAW_DATA"+'.xlsx',index=False)

(0, 4099)
No Data Avaiable or respective entries already imported
(198, 4099)
Data Import Successfully Completed
(198, 4099)
(207, 4099)
Data Import Successfully Completed
(207, 4099)
(192, 4099)
Data Import Successfully Completed
(192, 4099)
(192, 4099)
Data Import Successfully Completed
(192, 4099)
(0, 4099)
No Data Avaiable or respective entries already imported
(216, 4099)
Data Import Successfully Completed
(216, 4099)
(0, 4099)
No Data Avaiable or respective entries already imported
(180, 4099)
Data Import Successfully Completed
(180, 4099)
(156, 4099)
Data Import Successfully Completed
(156, 4099)
(0, 4099)
No Data Avaiable or respective entries already imported
(177, 4099)
Data Import Successfully Completed
(177, 4099)
(234, 4099)
Data Import Successfully Completed
(234, 4099)
(0, 4099)
No Data Avaiable or respective entries already imported
(171, 4099)
Data Import Successfully Completed
(171, 4099)
(0, 4099)
No Data Avaiable or respective entries already imported
(177, 4099)
Da

## get the date stored for each macid
macIds = IQNet_MacId_list()
for Id in list(macIds):
    last_extract_date = getDateTimeforMacId(Id)
    print(Id +': '+ str(last_extract_date))